In [ ]:
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

In [1]:
# Customize the following options!

# available options: htdemucs: first version of Hybrid Transformer Demucs. Trained on MusDB + 800 songs. Default model.
# htdemucs_ft: fine-tuned version of htdemucs, separation will take 4 times more time but might be a bit better. Same training set as htdemucs.
# htdemucs_6s: 6 sources version of htdemucs, with piano and guitar being added as sources. Note that the piano source is not working great at the moment.
# hdemucs_mmi: Hybrid Demucs v3, retrained on MusDB + 800 songs.
# mdx: trained only on MusDB HQ, winning model on track A at the MDX challenge.
# mdx_extra: trained with extra training data (including MusDB test set), ranked 2nd on the track B of the MDX challenge.
# mdx_q, mdx_extra_q: quantized version of the previous models. Smaller download and storage but quality can be slightly worse.
# SIG: where SIG is a single model from the model zoo.
model = "htdemucs" 

extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
# two_stems = None   # only separate one stems from the rest, for instance
two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = 'demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/' # Change to folder containing input songs files
out_path = 'demucs_output/' # Change to folder location containing output songs files

In [2]:
# Utility functions

def find_files(in_path, out_path) -> list:
    '''
    Finds the difference in audio files between input and output directory.
    Selects only the files only found in input folder. 
    '''
    input = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            input.append(file)
    done = []
    out_path = out_path + f'/{model}/'
    for file in Path(out_path).iterdir():
            done.append(file.name)
    final = []
    for file in input:
        if file.stem not in done:
            final.append(file)
    return final

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp, outp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


# def from_upload():
#     out_path = Path('separated')
#     in_path = Path('tmp_in')

#     if in_path.exists():
#         rmtree(in_path)
#     in_path.mkdir()

#     if out_path.exists():
#         rmtree(out_path)
#     out_path.mkdir()

#     uploaded = files.upload()
#     for name, content in uploaded.items():
#         (in_path / name).write_bytes(content)
#     separate(in_path, out_path)

In [3]:
separate(inp=in_path, outp=out_path)

Going to separate the files:
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Tuyết Rơi Mùa Hè.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Ăn Năn Muộn Màng.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Có Chó Nó Yêu Mày.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Vợ Tương Lai.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Lời Hẹn Mùa Xuân.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Mẹ Của Tôi.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Ai Mang Em Trở Lại.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Ân Tình Chia Đôi.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Tình Nhớ.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Chim Vành Khuyên.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Cướp Vợ.mp3
demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/T

/home/bernard/nus/cs5647/project/demucs/demucs/api.py:27: UserWarning: TorchAudio's global backend is now deprecated. Please enable distpatcher by setting `TORCHAUDIO_USE_BACKEND_DISPATCHER=1`, and specify backend when calling load/info/save function.
  import torchaudio as ta


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/bernard/nus/cs5647/project/demucs/demucs_output/htdemucs
Separating track demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Tuyết Rơi Mùa Hè.mp3


100%|██████████████████████████████████████████████████████████████████████| 274.95/274.95 [00:07<00:00, 35.15seconds/s]


Separating track demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Ăn Năn Muộn Màng.mp3


...

100%|██████████████████████████████████████████████████████████████████████| 274.95/274.95 [00:06<00:00, 42.37seconds/s]


Separating track demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Cùng Một Kiếp Hoa.mp3


100%|██████████████████████████████████████████████| 280.79999999999995/280.79999999999995 [00:06<00:00, 40.82seconds/s]


Separating track demucs_preprocessed_input/quoc_master_1st_1000_preprocessed/Em mang giấc mơ anh về đâu.mp3


100%|████████████████████████████████████████████████████████████████████████| 257.4/257.4 [00:06<00:00, 42.26seconds/s]
